# About
This notebook uses the feature extraction methods defined in features.py to create a .csv file of features for our image dataset.

It uses the "gan_augmented" data directory, which included symlinks from the original data PLUS the gan augmented data.

In [1]:
import sklearn

In [2]:
from tqdm import tqdm_notebook as tqdm

In [3]:
import numpy as np
import pandas as pd
import os
import glob

In [4]:
import features

In [5]:
DATA_DIR = "../data/gan_augmented"

In [6]:
train = os.path.join(DATA_DIR, "train")
test = os.path.join(DATA_DIR, "test")
train, test

('../data/gan_augmented/train', '../data/gan_augmented/test')

In [7]:
DATASET = test

In [ ]:
a = input("(!) STOP! Are you sure you want to run this Notebook? Previous files may be overwritten!! (y/N)")
if a != "y":
    assert False, "User chose to stop execution."

# Make feature names

In [8]:
import itertools
channels = ("gray", "alpha", "beta")
props = ("contrast", "dissimilarity", "homogeneity", "energy")
distances = ("1", "5")
degrees = ("0", "45", "90", "135")
feat_tuples = list(itertools.product(channels, props, distances, degrees))
feat_names = ["_".join(t) for t in feat_tuples]

for c in channels:
    last = "_".join((c, props[-1], distances[-1], degrees[-1]))
    index_insert = feat_names.index(last) + 1
    for morph_name in reversed(("num_elements", "average_compactness", "total_compactness")):
        feat_names.insert(index_insert, c + "_" + morph_name)

# Define function for iterating over patches

In [9]:
def patch_gen(image, size=400, step=1):
    for y in range(0, image.shape[0] - size + 1, int(size * step)):
        for x in range(0, image.shape[1] - size + 1, int(size * step)):
            yield image[y:y+size, x:x+size,:]

# Get files

In [10]:
from skimage import io
from typing import List
    
print("Running", DATASET)
files = glob.glob(DATASET + "/**/*.jpg", recursive=True) + glob.glob(DATASET + "/**/*.png", recursive=True)
files.sort()
len(files)
files[:10]

Running ../data/gan_augmented/test


['../data/gan_augmented/test/diestrus/C10_7_31_image2.jpg',
 '../data/gan_augmented/test/diestrus/C10_8_3_image1.jpg',
 '../data/gan_augmented/test/diestrus/C10_8_3_image2.jpg',
 '../data/gan_augmented/test/diestrus/C10_8_5_image2.jpg',
 '../data/gan_augmented/test/diestrus/C10_8_9_image2.jpg',
 '../data/gan_augmented/test/diestrus/C11_7_25_image2.jpg',
 '../data/gan_augmented/test/diestrus/C11_8_1_image2.jpg',
 '../data/gan_augmented/test/diestrus/C11_8_7_image2.jpg',
 '../data/gan_augmented/test/diestrus/C12_8_3_image1.jpg',
 '../data/gan_augmented/test/diestrus/C12_8_8_image1.jpg']

# Setup outfile

In [32]:
header = ",".join(["file", "label", "patch_num"] + feat_names) + "\n"

In [33]:
out_file = os.path.join(DATA_DIR, f"{os.path.basename(DATASET)}.csv")
out_file

'../data/gan_augmented/test.csv'

In [34]:
with open(out_file, "w") as f:
    f.write(header)

# Perform feature extraction

In [28]:
from joblib import Parallel, delayed

In [29]:
n_cpu = 4 # 8

In [30]:
def extract_features_from_images(files, out_file):
    for path in tqdm(files):
        label = path.split("/")[-2]

        img = io.imread(path)

        rows: List[str] = []

        for i, patch in enumerate(tqdm(list(patch_gen(img)), leave=False)):
            feat, _ = features.extract_delgado_features(patch)
            words = [os.path.basename(path), label, str(i)] + [str(f) for f in feat]
            row = ",".join(words)
            rows.append(row)

        with open(out_file, "a") as f:
            for r in rows:
                f.write(r + "\n")

In [31]:
# extract_features_from_images(files, out_file) # for this for single cpu run

In [18]:
def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))

file_splits = split(files, n_cpu)
[len(s) for s in file_splits]

[54, 54, 53, 53]

In [22]:
Parallel(n_cpu)(
    delayed(extract_features_from_images)(
        fil, out_file + f".{i}"
    ) for i, fil in tqdm(enumerate(file_splits))
)

[]

In [ ]:
for i in range(n_cpu):
    f = out_file + f".{i}"
    !cat $f >> $out_file
    !rm $f

In [ ]:
# copy the file as backup, in case this Notebook is run again.
!cp $out_file "$out_file".bak

# Read and visualize the created feature csv

In [ ]:
import pandas as pd
df = pd.read_csv(out_file)

In [ ]:
no_gan = df[df["file"].str.startswith("gan_")==False]

In [ ]:
gan_only = df[df["file"].str.startswith("gan_")]

In [ ]:
len(no_gan), len(gan_only)

In [ ]:
len(df)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = [15, 10]

no_gan.boxplot()

In [ ]:
gan_only.boxplot()

In [ ]:
df.boxplot()

In [ ]:
no_gan.hist()[0]

# Train a simple classifier

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = no_gan.drop(columns=["file", "label", "patch_num"])

In [ ]:
y = no_gan["label"]

In [ ]:
len(X), len(y)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

In [ ]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

In [ ]:
X_gan = gan_only.drop(columns=["file", "label", "patch_num"])
y_gan = gan_only["label"]

In [ ]:
X_train = np.append(X_train, X_gan, axis=0)
y_train = np.append(y_train, y_gan, axis=0)

In [ ]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

In [ ]:
from sklearn import svm

In [ ]:
clf = svm.SVC(verbose=True)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
predicted = clf.predict(X_val)

In [ ]:
accuracy = predicted == y_val

In [ ]:
accuracy.sum() / len(accuracy)

In [ ]:
from sklearn.neural_network import MLPClassifier
clf_mlp = MLPClassifier(hidden_layer_sizes=(22, 44), verbose=True)
clf_mlp.fit(X_train, y_train)

In [ ]:
predicted_mlp = clf_mlp.predict(X_val)
accuracy_mlp = predicted_mlp == y_val
accuracy_mlp.sum() / len(accuracy_mlp)